<a href="https://colab.research.google.com/github/KyungdaePark/HongongMachine/blob/master/Ensemble_of_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 앙상블 트리 中 랜덤 포레스트 기법
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('http://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine[['class']].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2, random_state = 42
)

In [ ]:
# RandomForestClassifier의 성능을 cross_validate로 교차검증 해보자
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(rf, train_input, train_target,
                        return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 기본 5-fold cv이므로 score는 5개가 나올거임. 그것들의 평균 np.mean

0.9973541965122431 0.8905151032797809


In [ ]:
# 결과를 보니 과대적합이 나옴.
# 랜덤 포레스트는 결정 트리의 앙상블이기 때문에 DecisionTreeClassifier가 제공하는 모든 매개변수 (criterion, max_depth ...)를 모두 제공함.
# 랜덤 포레스트가 사용한 특성 중요도를 출력해 보자.
rf.fit(train_input, train_target)
print(rf.feature_importances_)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


[0.23167441 0.50039841 0.26792718]


In [ ]:
# 기본 결정 트리에서는 중요도가 0.1 0.8 0.1 이었음. 랜덤 포레스트는 특성을 랜덤하게 선택해 여러 특성들이 훈련에 기여할 기회를 갖게 해줌.
# --> 과대적합을 막는데 도움이 됨

In [ ]:
# RandomForestClassifier에는 자체적으로 모델을 평가하는 기능이 있음.
# 훈련 세트에서 사용하지 않은 남는 샘플(Out Of Bag)들을 이용해 자체 검증이 가능.
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


0.8934000384837406


In [ ]:
# 교차 검증 결과 : 
print(np.mean(scores['test_score']))
# 와 매우 비슷함.

0.8905151032797809


In [ ]:
##############################################################
# 다음은 엑스트라 트리. 엑스트라 트리는 랜덤 포레스트와 매우 비슷하게 작동함.
# DecisionTreeClassifier의 매개변수 역시 동일하게 사용 가능함.
# 랜덤 포레스트와의 차이라면 훈련/테스트 세트를 만들때 부트스트랩 샘플을 사용하지 않음. 즉 결정트리를 만들때 모든 훈련 세트를 사용함.
# 대신 노드 분할 시 최적의 분할을 찾지 않고 무작위로 분할함.
# DecisionTreeClassifier의 매개변수 splitter="random"인 트리가 엑스트라트리가 사용하는 결정트리임.
# 장점 : 많은 트리를 앙상블하기 때문에 과대적합↓ 검증 세트의 점수↑ 속도↑

from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
# 특성 중요도 :
et.fit(train_input, train_target)
print(et.feature_importances_)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


[0.20183568 0.52242907 0.27573525]


In [ ]:
############################################################
# 그레이디언트 부스팅 (gradient boosting)
# 깊이가 얕은 결정트리를 이용함. 기본 깊이 3
# 경사 하강법을 앙상블에 추가함
# 경사 하강법은 모델의 가중치와 절편을 조금씩 바꾸는 과정
# 경사 하강법은 조금씩 천천히 움직여야 함
# 그래서 여기서도 깊이가 3인 얕은 트리를 사용한다.

from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, n_jobs = -1, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))


0.8881086892152563 0.8720430147331015


In [ ]:
# 과대적합이 거의 일어나지 않음. 결정트리의 개수를 늘려도 과대적합에 강하다.
# 학습률을 증가시키고 트리의 개수를 늘리면 성능이 향상됨
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state = 42) #learning_rate 의 기본값은 0.1
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.15872278 0.68010884 0.16116839]


In [ ]:
# subsample의 매개변수를 1보다 작게하면 훈련 세트의 일부만을 이용해 훈련함 "확률적/미니배치 경사 하강법"
# 지금은 배치 경사 하강법

In [ ]:
######################################################################
# 히스토그램 기반 그레디언트 부스팅 (Histogram-based Gradient Boosting)
# 그레디언트 부스팅은 게속 트리를 추가하기 때문에 훈련속도가 느린데, 이를 보안한게 히스토그램 기반 그레디언트 부스팅임
# 특성을 256개의 구간으로 나누어 최적의 분할을 매우 빠르게 처리함.

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input ,train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# Gradient Boosting의 트리의 개수를 지정했던 n_estimators 대신 HistGradientBoostingClassifier는 max_iter 를 조정하여 부스팅 반복 횟수를 지정함

/usr/local/lib/python3.7/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed whe

0.9321723946453317 0.8801241948619236


In [ ]:
hgb.fit(train_input, train_target)
# print(hgb.feature_importances_) ERROR

# 특성 중요도를 알아보자
# permutation_importance()를 사용함
# 특성을 무작위로 섞어서 어떤 특성이 들어갔을 때 제일 성능이 좋은지를 판단해 그때의 특성을 출력함
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs=-1)
print(result.importances_mean)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.08876275 0.23438522 0.08027708]


In [ ]:
# 랜덤 포레스트와 비슷한 비율이다.
# rf = [0.23167441 0.50039841 0.26792718]
# 테스트 세트에서의 특성 중요도 ?
result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923